In [17]:
# Import neccessary libraries
from datetime import datetime
import pandas as pd
import numpy as np

!pip install metpy
from metpy.io import parse_metar_to_dataframe
import glob

In [3]:
 from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Read in the data
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2001/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2002/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2003/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2004/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2005/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2006/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2007/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2008/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2009/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2010/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2011/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2012/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2013/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2014/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2015/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2016/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2017/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2018/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2019/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2020/64010KMSN*.dat'

--2020-04-21 19:36:09--  ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KMSN*.dat
           => ‘.listing’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.101, 2610:20:8040:2::101
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.101|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/asos-fivemin/6401-2000 ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [     <=>            ] 224.33K   129KB/s    in 1.7s    

2020-04-21 19:36:13 (129 KB/s) - ‘.listing’ saved [229713]

Removed ‘.listing’.
--2020-04-21 19:36:13--  ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KMSN200001.dat
           => ‘64010KMSN200001.dat’
==> CWD not required.
==> PASV ... done.    ==> RETR 64010KMSN200001.dat ... done.
Length: 1139705 (1.1M)

64010KMSN200001.dat 100%[===================>]   1.09M   344KB/s    in 3.2s    

2020-04-21 19:36:17 (34

In [0]:

def parse_metar_file(file, wx_subset=True):
    """
    Parses METAR file from NCDC

    Input:
    --------
    file = Text file downloaded from NCDC

    wx_subset = Flag to determine whether or not to drop non-current weather obs (if True, only returns obs with observed weather)

    Output:
    --------
    df = Pandas dataframe filtered for times where current weather is not 'nan'
    """

    # Read in the file using Pandas
    df = pd.read_csv(file, header=None)

    # Pull the timestamp from the filename
    timestamp = datetime.strptime(file[-10:], '%Y%m.dat')

    # Iterrate over rows to parse METARS
    df_list = []
    for index, row in df.iterrows():
        try:
            df_list.append(parse_metar_to_dataframe(row.values[0][52:], year=timestamp.year, month=timestamp.month))
        except:
            print('Error with METAR: ', row.values[0][52:])
    #
    merged_df = pd.concat(df_list)

    # Drop datasets that do not include current weather
    merged_df = merged_df.dropna(subset=['current_wx1'])

    # Change the index to datetime
    merged_df.index = merged_df.date_time

    # Return the merged dataset sorted by datetime
    return merged_df.sort_index()




In [0]:
# Find all files in the data directory
files = sorted(glob.glob('*.dat'))

# Create a list to store the dataframes in
merged_datasets = []


In [8]:

# Loop through and parse the different datasets
for file in files:
    print(file[-10:])
    try:
        merged_datasets.append(parse_metar_file(file))
    except:
        print("Error with :", file)



200001.dat


/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


200002.dat
200003.dat
200004.dat
200005.dat
Error with : 64010KMSN200005.dat
200006.dat
Error with METAR:   23:40:31  5-MIN KMSN 140540Z 25010G16KT 10SM FEW012 BKN022 OVC06520/18 A2970 1070 87 1900 240/10G16 RMK AO2 CIG 017V033 $
Error with METAR:   KMSN 162
Error with METAR:  06/16/00 09:15:31  5-MIN KMSN 161515Z 26009G17KT 200V290 10SM FEW028 SCT034  17/12 A2962 1140 69 1700 260/09G17 200V290 RMK AO2
Error with METAR:   KMSN 209
Error with METAR:   KMSN 209 A2965 1110 64 2700 220/12G18 RMK KMSN 202250Z 23010G18KT 10SM FEW036 BKN06/20/00
Error with METAR:  MK AO2
Error with METAR:  R 23/10 A2973 1006/21/00 10:05:31  5-MINR 23/11
Error with METAR:  A2973 1040
Error with METAR:  R 23/10
Error with METAR:   KMSN 2206/21/00 18:35:31  5-MIN KMSN 220035Z 22011KT 10SM CLR 23/11 A2966 1110 44 2400 220/11 RMK AO2
Error with METAR:  1  5-MIN KMSN 220125Z 22010G16KT 10SM CLR 22/11 A2966 111047 2200 220/10G16 RMK AO2
Error with METAR:  
Error with METAR:   KMSN 22310/08 RMK AO2
Error with METAR: 

In [10]:
# Return the cleaned dataset
#clean_df = (merged_datasets)
decoded_metar_master = pd.concat(merged_datasets)

## Save the file to memory
#clean_df.to_csv('decoded_metar_dataset.csv')
# Print out the resulting dataset
decoded_metar_master

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,current_wx1,current_wx2,current_wx3,low_cloud_type,low_cloud_level,medium_cloud_type,medium_cloud_level,high_cloud_type,high_cloud_level,highest_cloud_type,highest_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,air_pressure_at_sea_level,eastward_wind,northward_wind
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:50:00,KMSN,43.13,-89.35,261,2000-01-01 00:50:00,NaN,5.0,-SN,NaN,NaN,FEW,2000,OVC,2700.0,NaN,NaN,NaN,NaN,8,-6.0,-9.0,29.92,71,0,0,NaN,NaN,NaN
2000-01-01 00:55:00,KMSN,43.13,-89.35,261,2000-01-01 00:55:00,320.0,6.0,-SN,NaN,NaN,BKN,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-6.0,-9.0,29.92,71,0,0,NaN,3.856726,-4.596267
2000-01-01 01:00:00,KMSN,43.13,-89.35,261,2000-01-01 01:00:00,340.0,7.0,-SN,NaN,NaN,SCT,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-5.0,-9.0,29.92,71,0,0,NaN,2.394141,-6.577848
2000-01-01 01:05:00,KMSN,43.13,-89.35,261,2000-01-01 01:05:00,330.0,10.0,-SN,NaN,NaN,SCT,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-5.0,-10.0,29.92,71,0,0,NaN,5.000000,-8.660254
2000-01-01 01:10:00,KMSN,43.13,-89.35,261,2000-01-01 01:10:00,320.0,8.0,-SN,NaN,NaN,FEW,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-6.0,-10.0,29.93,71,0,0,NaN,5.142301,-6.128356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30 00:25:00,KMSN,43.13,-89.35,261,2020-03-30 00:25:00,290.0,7.0,-RA,NaN,NaN,FEW,1900,SCT,2500.0,OVC,3600.0,NaN,NaN,8,5.0,2.0,29.77,61,0,0,NaN,6.577848,-2.394141
2020-03-30 00:30:00,KMSN,43.13,-89.35,261,2020-03-30 00:30:00,280.0,8.0,-RA,NaN,NaN,FEW,1900,BKN,2500.0,OVC,3400.0,NaN,NaN,8,5.0,2.0,29.77,61,0,0,NaN,7.878462,-1.389185
2020-03-31 16:55:00,KMSN,43.13,-89.35,261,2020-03-31 16:55:00,40.0,9.0,-SN,NaN,NaN,OVC,1300,NaN,NaN,NaN,NaN,NaN,NaN,8,2.0,-1.0,30.08,71,0,0,NaN,-5.785088,-6.894400


In [11]:
# Create month column and add to the dataframe
time = pd.to_datetime(decoded_metar_master['date_time'])
decoded_metar_master['month'] = time.dt.month
decoded_metar_master

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,current_wx1,current_wx2,current_wx3,low_cloud_type,low_cloud_level,medium_cloud_type,medium_cloud_level,high_cloud_type,high_cloud_level,highest_cloud_type,highest_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,air_pressure_at_sea_level,eastward_wind,northward_wind,month
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:50:00,KMSN,43.13,-89.35,261,2000-01-01 00:50:00,NaN,5.0,-SN,NaN,NaN,FEW,2000,OVC,2700.0,NaN,NaN,NaN,NaN,8,-6.0,-9.0,29.92,71,0,0,NaN,NaN,NaN,1
2000-01-01 00:55:00,KMSN,43.13,-89.35,261,2000-01-01 00:55:00,320.0,6.0,-SN,NaN,NaN,BKN,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-6.0,-9.0,29.92,71,0,0,NaN,3.856726,-4.596267,1
2000-01-01 01:00:00,KMSN,43.13,-89.35,261,2000-01-01 01:00:00,340.0,7.0,-SN,NaN,NaN,SCT,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-5.0,-9.0,29.92,71,0,0,NaN,2.394141,-6.577848,1
2000-01-01 01:05:00,KMSN,43.13,-89.35,261,2000-01-01 01:05:00,330.0,10.0,-SN,NaN,NaN,SCT,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-5.0,-10.0,29.92,71,0,0,NaN,5.000000,-8.660254,1
2000-01-01 01:10:00,KMSN,43.13,-89.35,261,2000-01-01 01:10:00,320.0,8.0,-SN,NaN,NaN,FEW,2700,OVC,3300.0,NaN,NaN,NaN,NaN,8,-6.0,-10.0,29.93,71,0,0,NaN,5.142301,-6.128356,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30 00:25:00,KMSN,43.13,-89.35,261,2020-03-30 00:25:00,290.0,7.0,-RA,NaN,NaN,FEW,1900,SCT,2500.0,OVC,3600.0,NaN,NaN,8,5.0,2.0,29.77,61,0,0,NaN,6.577848,-2.394141,3
2020-03-30 00:30:00,KMSN,43.13,-89.35,261,2020-03-30 00:30:00,280.0,8.0,-RA,NaN,NaN,FEW,1900,BKN,2500.0,OVC,3400.0,NaN,NaN,8,5.0,2.0,29.77,61,0,0,NaN,7.878462,-1.389185,3
2020-03-31 16:55:00,KMSN,43.13,-89.35,261,2020-03-31 16:55:00,40.0,9.0,-SN,NaN,NaN,OVC,1300,NaN,NaN,NaN,NaN,NaN,NaN,8,2.0,-1.0,30.08,71,0,0,NaN,-5.785088,-6.894400,3


In [0]:
#decoded_metar_master = decoded_metar_master.dropna(axis=0, subset=['current_wx1','air_temperature'])

## Drop columns we don't plan on using or have too many NaNs
decoded_metar_master = decoded_metar_master.drop(['low_cloud_type',	'low_cloud_level',	'medium_cloud_type',	'medium_cloud_level',	'high_cloud_type',	'high_cloud_level',	'highest_cloud_type',	'highest_cloud_level',	'cloud_coverage', 'air_pressure_at_sea_level'], axis=1)

#decoded_metar_master = decoded_metar_master.dropna(axis=0)

In [13]:
#convert current_wx columns to list and also check how many rows we have 
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))
current_wx2 = decoded_metar_master['current_wx2'].to_list()
print(len(current_wx2))
current_wx3 = decoded_metar_master['current_wx3'].to_list()
print(len(current_wx3))

#Check to make sure we have some frozen AND liquid weather reported
print(current_wx1[0:50])

418077
418077
418077
['-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', 'HZ', '-SN', '-SN', '-SN', '-RA', '-RA', '-RA', 'RA', '-RA', '-RA', '-RA', 'BR', 'BR', 'BR']


In [14]:
# List all of the current weather identifiers to identify non-precipitation identifiers to be removed
print(decoded_metar_master.current_wx1.unique())

['-SN' 'HZ' '-RA' 'RA' 'BR' '-SNRA' 'FG' '-FZDZ' 'SN' 'BL' '-FZRA' 'FZRA'
 '+RA' 'TS' '+SN' '-TSRA' 'TSRA' '+TSRA' 'VCTS' 'MIFG' 'FZFG' '-SNDZ'
 'VCFG' '+TS' '-TS' 'FZDZ' '-TSDZ']


In [0]:
# Create non-precip variables list to replace with NaNs
non_precip = ['HZ', 'BR', 'FG', '-FZDZ', 'BL', 'TS', 'VCTS', 'MIFG', 'FZFG', '-SNDZ', 'VCFG', '+TS', '-TS', 'FZDZ', '-TSDZ']

In [18]:
# Replace non-precip variables with NaNs
decoded_metar_master.replace(non_precip, np.nan, inplace=True)
### Then drop current_wx2 and current_wx3 because they hold a lot of NaNs
decoded_metar_master = decoded_metar_master.drop(['current_wx2',	'current_wx3'], axis=1)
### Then drop all other rows with NaNs
decoded_metar_master = decoded_metar_master.dropna(axis=0)
### Check how many rows we are left with after cleaning up the dataframe
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))

178094


In [0]:
### List of frozen and liquid precip options
fzn_list = ['-SNRA','SNRA','+SNRA','-SN','SN','+SN','-FZRA','FZRA','+FZRA', '-TSSN', 'TSSN', '+TSSN']
liq_list = ['-RA','RA','+RA','-TS','TS','+TS','-TSRA','TSRA','+TSRA']

In [0]:
### Set up a list to hold binary values (1 for frozen, 0 for liquid)
binary_list = []

### Loop over current_wx observations and determin if any are frozen precip. If so, set binary variable to 1. If not, set binary variable to 0.
for i in range(len(current_wx1)):
    if (current_wx1[i] in fzn_list) or (current_wx2[i] in fzn_list) or (current_wx3[i] in fzn_list):
        binary_list.append(1)
    else:
        binary_list.append(0)

### Fill a new column called 'fzn_or_liq' with the binary values.
decoded_metar_master['fzn_or_liq'] = binary_list


In [21]:
# Double check remaining parameters are correct and all precipitation
decoded_metar_master.current_wx1.unique()

array(['-SN', '-RA', 'RA', '-SNRA', 'SN', '+RA', '+SN', '-FZRA', '-TSRA',
       'TSRA', '+TSRA', 'FZRA'], dtype=object)

In [22]:
decoded_metar_master

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,current_wx1,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,eastward_wind,northward_wind,month,fzn_or_liq
date_time,,,,,,,,,,,,,,,,,,
2000-01-01 00:55:00,KMSN,43.13,-89.35,261,2000-01-01 00:55:00,320.0,6.0,-SN,-6.0,-9.0,29.92,71,0,0,3.856726,-4.596267,1,1
2000-01-01 01:00:00,KMSN,43.13,-89.35,261,2000-01-01 01:00:00,340.0,7.0,-SN,-5.0,-9.0,29.92,71,0,0,2.394141,-6.577848,1,1
2000-01-01 01:05:00,KMSN,43.13,-89.35,261,2000-01-01 01:05:00,330.0,10.0,-SN,-5.0,-10.0,29.92,71,0,0,5.000000,-8.660254,1,1
2000-01-01 01:10:00,KMSN,43.13,-89.35,261,2000-01-01 01:10:00,320.0,8.0,-SN,-6.0,-10.0,29.93,71,0,0,5.142301,-6.128356,1,1
2000-01-01 01:15:00,KMSN,43.13,-89.35,261,2000-01-01 01:15:00,320.0,8.0,-SN,-6.0,-10.0,29.93,71,0,0,5.142301,-6.128356,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30 00:25:00,KMSN,43.13,-89.35,261,2020-03-30 00:25:00,290.0,7.0,-RA,5.0,2.0,29.77,61,0,0,6.577848,-2.394141,3,0
2020-03-30 00:30:00,KMSN,43.13,-89.35,261,2020-03-30 00:30:00,280.0,8.0,-RA,5.0,2.0,29.77,61,0,0,7.878462,-1.389185,3,0
2020-03-31 16:55:00,KMSN,43.13,-89.35,261,2020-03-31 16:55:00,40.0,9.0,-SN,2.0,-1.0,30.08,71,0,0,-5.785088,-6.894400,3,1


In [0]:
# Save the final data to a csv file
decoded_metar_master.to_csv('/content/drive/My Drive/decoded_metar_FINAL.csv')